In [2]:
from datasets import load_dataset
from pathlib import Path
import subprocess

/Volumes/mySSD/dev/typst-OCR/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset(
    "MixTex/Pseudo-Latex-ZhEn-1",
    cache_dir="/Volumes/mySSD/"
)

In [4]:
img = ds["train"][0]["image"]
img.show()

In [5]:
ds["train"]

Dataset({
    features: ['image', 'text'],
    num_rows: 117738
})

In [6]:
ds["train"]['text'][12:13]

['\n \nX] 是有限\n \\[ \n&sym :\\mathbb{D} \\times \\mathbb{D} \\rightarrow \\mathbb{C} \\times \\mathbb{C}\\\\&\\underline{z}=(z_1, z_2) \\mapsto (z_1 +z_2, z_1 z_2).\n \\] \n生成数的 \\(1.0\\) 情况下，我们给出一个算法，用于找到一个准\n \\[ \n \\begin{cases*} \\partial_{\\mu} (m^{\\mu \\nu}\\alpha\\partial_{\\nu}\\Psi)=F(\\Psi,\\partial \\Psi)\\\\ (\\Psi,\\partial_t \\Psi)|_{\\{t=0\\}}=(\\Psi_0, \\Psi_1) \\in \\mathcal{H}. \\end{cases*} \n \\] \n实数域 K[X]，使得它的坐标域是 G 的有限生成数。在发展过程中，我们 \\textit{Diospyros} 还介绍了一些处理 \\(k<0\\) 非有限生成数的算法。特别地，我们引入了有限生成域理想。我们使用一系列N-体模拟LCDM宇宙学\n \n']

In [7]:
import re

pattern = re.compile(
    r"#math\.equation\s*\(\s*"
    r"block\s*:\s*(true|false)\s*,\s*"
    r"\$(.*?)\$\s*"
    r"\)",
    re.IGNORECASE | re.DOTALL
)

def replace_math(match):
    block = match.group(1).lower() == "true"
    content = match.group(2).strip()

    if block:
        return f"$ {content} $"
    else:
        return f"${content}$"

In [8]:
from copy import deepcopy
import tempfile
ds_typ = deepcopy(ds)

In [12]:
# 跑需要约20分钟
texts = ds_typ["train"]["text"]
typst_results = []
failed_count = 0

for i, latex_text in enumerate(texts):
    if (i % 100 == 0):
        print(f"处理第 {i} 条...")

    # 每一条用独立的临时文件
    with tempfile.TemporaryDirectory() as tmpdir:
        tex_path = Path(tmpdir) / "input.tex"
        typ_path = Path(tmpdir) / "output.typ"

        # 写入 LaTeX 内容
        tex_path.write_text(latex_text, encoding="utf-8")

        # 调用 mitex 编译
        try:
            subprocess.run(
                ["mitex", "compile", str(tex_path), str(typ_path)],
                check=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
            )

            # 编译成功，读取输出文件
            if typ_path.exists():
                # 格式化
                subprocess.run(
                    ["/Users/hzy/Downloads/typstyle", "-i", str(typ_path)],
                    check=True,
                )
                raw = typ_path.read_text(encoding="utf-8")
                # 分行 → 切片去掉前144行 → 再拼回去
                lines = raw.splitlines()
                trimmed = "\n".join(lines[144:]) if len(lines) > 144 else ""
                # 去除`#math.equation()`
                trimmed = pattern.sub(replace_math, trimmed)

                typst_results.append(trimmed.strip())
            else:
                typst_results.append(None)

        except subprocess.CalledProcessError as e:
            typst_results.append(None)
            failed_count += 1

print(len(typst_results))
# 将结果写入 Dataset
ds["train"] = ds["train"].add_column("Typst", typst_results)
print("completed, failed count:", failed_count)

处理第 0 条...


warn: Failed to parse /var/folders/r7/70_c2qpd0_71wl1p5srk_s1c0000gn/T/tmpuxw6ehz0/output.typ. The source is erroneous.
warn: Failed to parse /var/folders/r7/70_c2qpd0_71wl1p5srk_s1c0000gn/T/tmp9d5q2j0g/output.typ. The source is erroneous.
warn: Failed to parse /var/folders/r7/70_c2qpd0_71wl1p5srk_s1c0000gn/T/tmpqtxjk73l/output.typ. The source is erroneous.


处理第 100 条...
处理第 200 条...
处理第 300 条...
处理第 400 条...
处理第 500 条...
处理第 600 条...
处理第 700 条...
处理第 800 条...


warn: Failed to parse /var/folders/r7/70_c2qpd0_71wl1p5srk_s1c0000gn/T/tmpt09q9kc_/output.typ. The source is erroneous.


处理第 900 条...
处理第 1000 条...
处理第 1100 条...
处理第 1200 条...
处理第 1300 条...
处理第 1400 条...
处理第 1500 条...
处理第 1600 条...
处理第 1700 条...


...

处理第 117200 条...


warn: Failed to parse /var/folders/r7/70_c2qpd0_71wl1p5srk_s1c0000gn/T/tmprqyagek7/output.typ. The source is erroneous.


处理第 117300 条...
处理第 117400 条...


warn: Failed to parse /var/folders/r7/70_c2qpd0_71wl1p5srk_s1c0000gn/T/tmpzso_iuej/output.typ. The source is erroneous.


处理第 117500 条...
处理第 117600 条...
处理第 117700 条...
117738
completed, failed count: 6538


In [13]:
# 剔除所有Typst为空的行，即转换失败的行
ds["train"] = ds["train"].filter(lambda x: x["Typst"] is not None)

Filter: 100%|██████████| 117738/117738 [00:39<00:00, 2991.04 examples/s]


In [14]:
train_ds = ds["train"]

train_ds.to_parquet("/Volumes/mySSD/pseudo_typst_train_clean.parquet")

Creating parquet from Arrow format: 100%|██████████| 1112/1112 [00:15<00:00, 74.06ba/s] 


1284165874

In [9]:
train_ds = load_dataset(
    "parquet",
    data_files="/Volumes/mySSD/pseudo_typst_train_clean.parquet"
)

In [10]:
train_ds["train"][0]["image"].show()
print(train_ds["train"][0]["Typst"])

$2$; 态类对称函数 $mitexmathbf(J)$; 的泛函域结构。我们研究了相同框架中一般线性群的直角子群和 $9 1 3$; 复数子群
$
  aligned(\(omega + i diff macron(diff) w \)^(n + 1 )&= 0 wide macron(D)times X \ omega + i diff macron(diff) w | \{ s \} times X &> 0 s in macron(D)\,)
$
的一般线性群。我们证明了  $f V$; CSp 和 CSp 也具有与 CGL 相似的等价泛函域结构，并且具有与 CGL 相似 $1$; 的算子结构。这些泛函域的定义与标准基有关，类似于对称函


In [20]:
from transformers import AutoTokenizer, VisionEncoderDecoderModel, AutoImageProcessor
tokenizer = AutoTokenizer.from_pretrained("MixTex/ZhEn-Latex-OCR", max_len=296)
vocab_dict = tokenizer.get_vocab()
print(f"Vocabulary size: {len(vocab_dict)}")

Vocabulary size: 25679


In [ ]:
count=0
for i in vocab_dict:
    print(i)

ĠDirac
Ġprogramming
lli
dN
åºŀ
{\,
Ġplane
|
!(
è´¨åŃĲ
é£Łåĵģ
çļĦä»£æķ°
æįĲ
æ©ĺ
|\\&\
çĽ´
æºĲèĩª
).
åľ¨æŁĲ
æīĢå¾Ĺ
éĥ½åı¯
çļĦåħ¶ä»ĸ
æ¢°
ĠæĬĢæľ¯
Ġlonger
ç¥ŀåľ£ç½Ĺé©¬å¸ĿåĽ½
ç»ĺåĪ¶
åħ³éĹŃ
equation
ç®Ń
ç¢³
Ġshot
Ġvectors
}+
intu
åĴĮæĸĩåĮĸ
å¹³éĿ¢ä¸Ĭ
)\,.
çĵ·
åĲĽä¸»
åŀĤçĽ´äºİ
Ġreasonably
Ġtaking
sso
Ġan
éĩıåº¦
At
rating
Ġinelastic
259
Ġgrain
çļĦå¤©
Ġkinds
nx
}\|
Ġgeneralize
Á
Ġtiming
çļĦæĹĭè½¬
Ġçī¹å¾ģ
ç¼ĸåĨĻ
ĠåĲį
107
å·¥åİĤ
pt
Omega
Ġentanglement
cm
Ġprecisely
Ġarrays
Ġäºĭä»¶
alog
ä¸ĭåİ»
)|^{
when
nl
å¹´ä»¥åīį
ymbol
åĽ¾åĥı
å¯Ĩçłģ
Ġcooperation
æł
Ġ\|_{
Ġexploited
æĪĲç«ĭäºĨ
æĵįçºµ
Ġfavor
Ġbath
pv
åłªèĲ¨æĸ¯
ä¼½
statis
éĩı
pies
Ġabove
ĠåĮºåŁŁ
arent
open
åĢĴ
Ġprotostellar
åĢįæķ°
Ġquiescent
Ġfiel
åħ¶åŃĲ
}^{*}(
åħ±è½Ń
Ġtakes
çļĦå½¢æĪĲ
åıĮåĲĳ
priate
æµĭå¾Ĺ
Ġaffected
æĶ¿åħļ
çĹķ
ĠCat
æıĲä¾ĽäºĨä¸Ģä¸ª
Ġprinci
Ġerg
Ġexperience
ĠçŁ
Ġcontrol
xing
ĠPop
Ġlast
äºĶä¸ĸ
Ġcircle
Ġenvelop
éĩıåŃĲåĬĽåŃ¦ä¸Ń
dents
Ġnonmagnetic
Ġnoninteracting
Ġç©ºéĹ´çļĦ
Ġäºī
sche
Ġwhereas
Ġç¬¬äºĮ
Ġeven
Ġabsorbing
ä½¿ä»ĸ
mber
pop
å¾®æ³¢


In [ ]:
# id -> token
id2token = [None] * len(vocab_dict)
for tok, idx in vocab_dict.items():
    id2token[idx] = tok

# 查看前 200 个
for i in range(500,600):
    print(i, repr(id2token[i]))

500 'æĪĳä»¬'
501 'Ġco'
502 'äºİ'
503 'å½'
504 'åĪĨ'
505 'ng'
506 'çĶ¨'
507 'xi'
508 'Ġbe'
509 'Ġthat'
510 'mega'
511 'Ġwith'
512 'Ġon'
513 'Ġ0'
514 'æł'
515 'Ġex'
516 'ted'
517 'çİ'
518 'å¯¹'
519 'æī'
520 'partial'
521 'ĠS'
522 'æµ'
523 'tic'
524 'ur'
525 'è¿Ļ'
526 'èĢ'
527 'éĹ'
528 'ine'
529 '_\\'
530 'Ġan'
531 'ĠI'
532 'igma'
533 'to'
534 'end'
535 'ĥ½'
536 'Ġmo'
537 'åı¯'
538 'pe'
539 'Ġ\\)'
540 'ĠWe'
541 'dots'
542 'å¹´'
543 'ab'
544 'åŃĲ'
545 'fty'
546 'infty'
547 'æĢ§'
548 'res'
549 'Ġh'
550 'beta'
551 'å¤§'
552 '}{\\'
553 'dot'
554 'ĠC'
555 '}='
556 'per'
557 '},'
558 'di'
559 'mi'
560 'Ġ1'
561 ')\\)'
562 'lon'
563 'åº¦'
564 'rm'
565 'be'
566 'ãĢģ'
567 'gamma'
568 '),'
569 'éĿ'
570 'ss'
571 'theta'
572 'æŃ'
573 'ĠM'
574 'å·'
575 'ĠThe'
576 'æĭ'
577 'time'
578 'big'
579 'ä¸į'
580 'pp'
581 'sigma'
582 'nu'
583 'til'
584 'ĠF'
585 'ngle'
586 'Ġwe'
587 'as'
588 'ĠH'
589 'æĸ¹'
590 'Ġare'
591 'ci'
592 'Ġal'
593 '|\\'
594 'çĲ'
595 'tu'
596 'line'
597 'è´'
598 'ĠB'
599 'ĠL'


In [ ]:
for i, t in enumerate(id2token[500:510]):
    try:
        print(i+500, t, "->", t.encode("latin1"))
    except Exception as e:
        print(i+500, t, e)

500 æĪĳä»¬ 'latin-1' codec can't encode characters in position 1-2: ordinal not in range(256)
501 Ġco 'latin-1' codec can't encode character '\u0120' in position 0: ordinal not in range(256)
502 äºİ 'latin-1' codec can't encode character '\u0130' in position 2: ordinal not in range(256)
503 å½ -> b'\xe5\xbd'
504 åĪĨ 'latin-1' codec can't encode characters in position 1-2: ordinal not in range(256)
505 ng -> b'ng'
506 çĶ¨ 'latin-1' codec can't encode character '\u0136' in position 1: ordinal not in range(256)
507 xi -> b'xi'
508 Ġbe 'latin-1' codec can't encode character '\u0120' in position 0: ordinal not in range(256)
509 Ġthat 'latin-1' codec can't encode character '\u0120' in position 0: ordinal not in range(256)


: 